In [1]:
# use with local installs that don't have nbdev
import sys

sys.path.insert(0, "../../")


In [2]:
# pip install mdutils


In [3]:
# pip install markdownify

In [4]:
URL = "https://domo-support.domo.com/s/article/360047400753?language=en_US"
BASE_URL = "https://domo-support.domo.com/"

In [5]:
import datacrew.crawler.crawler as dcc

driver = dcc.driversetup(is_headless=False)


In [6]:
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.by import By

import time
from bs4 import BeautifulSoup

import datacrew.crawler.crawler as dcc

test_page_source = dcc.pagesource(url = URL, driver = driver , element_type= By.CLASS_NAME, element_id= "slds-form-element")
# test_page_source

💤 loading https://domo-support.domo.com/s/article/360047400753?language=en_US 💤
Page https://domo-support.domo.com/s/article/360047400753?language=en_US is loaded within 10 seconds.


c:\Users\jwilson1\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\crawler.py:106: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 106 of the file c:\Users\jwilson1\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\crawler.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(driver.page_source)


In [7]:
from dataclasses import dataclass, field
import urllib.parse as url_parse

from bs4 import BeautifulSoup
import selenium.webdriver
import markdown as md


@dataclass
class Article:
    base_url: str
    soup: BeautifulSoup = field(repr=False, default=None)
    linked_url_ls: list[str] = field(default_factory=list)
    images_ls: list[str] = field(default_factory=list)

    def __post_init__(self):
        self.get_linked_urls()

    @classmethod
    def get_from_url(cls, url: str, driver, base_url: str):
        soup = dcc.pagesource(
            driver=driver, url=url, element_type=By.CLASS_NAME, element_id="slds-form-element")

        return cls(soup=soup, base_url=base_url)

    @staticmethod
    def md_soup(soup, **options):
        return md.MarkdownConverter(**options).convert_soup(soup)

    def get_linked_urls(self, is_remove_query_string_parameters: bool = True):
        self.linked_url_ls = []
        for soup_link in self.soup.find_all("a"):
            url = soup_link.get("href")

            if not url:
                continue

            if url.startswith("/"):
                url = url_parse.urljoin(self.base_url, url)

            if is_remove_query_string_parameters:
                url = url_parse.urljoin(url, url_parse.urlparse(url).path)

            if url.startswith(self.base_url) and url not in self.linked_url_ls:
                self.linked_url_ls.append(url)

        return self.linked_url_ls

    def get_images(self, test_base_url: str = None , debug_prn: bool = False):
        self.image_ls = [{
            "url": f"{self.base_url}{item.get('src')}",
            "relative_url": item.get('src'),
            "name": item.get('alt') } for item in self.soup.find_all('img')]

        if test_broken_url:
           self.image_ls = [img for img in self.image_ls if img.get('url').startswith(test_base_url)]
                
        if debug_prn:
            print(self.image_ls)
        return self.image_ls


# test_art = Article.get_from_url(url = URL, driver = driver, base_url = base_url)
# test_art


In [8]:
import re
import datetime as dt
import markdownify as md
from dataclasses import dataclass, field
from dateutil import parser

# @dataclass
# class Article:
#   soup: BeautifulSoup = field(repr = False)
#   base_url:str

class ArticleKB_GetSoupError:
    def __init__(self, url):
        super().__init__(f"failed to retrieve soup for {url}")

class ArticleKB_ProcessSoupError(Exception):
    def __init__(self, url, search_term):
        super().__init__(f"search term {search_term} does not exist in {url}")


@dataclass(init=False)
class Article_KB(Article):
    url: str
    base_url: str
    driver: selenium.webdriver

    is_success:bool = False
    article: Article = field(default=None, repr=False)
    kb_soup: BeautifulSoup = field(default=None, repr=False)
    kb_url_ls: list[str] = field(default=None)

    title: str = None
    md_str: str = field(default=None, repr=False)
    article_id: str = None
    views: int = None
    created: dt.date = None
    last_updated: dt.date = None

    def __init__(self, url, base_url, driver):
        self.url = url
        self.base_url = base_url
        self.driver = driver

        soup = dcc.pagesource(driver=self.driver, url=self.url,
                              element_type=By.CLASS_NAME, element_id="slds-form-element")

        if not soup:
            raise ArticleKB_GetSoupError(url = self.url)

        super().__init__(base_url = base_url, soup = soup)

        self.article = Article(soup=soup, base_url=self.base_url)
        self.kb_url_ls = self.article.linked_url_ls
        
        try:
            self.process_kb_soup(soup)
            self.is_success = True

        except ArticleKB_ProcessSoupError as e:
            print(e)

        

    def process_kb_soup(self, soup: BeautifulSoup):
        search_term = "slds-form-element"

        table = soup.find_all(class_=[search_term])

        if not table or table == []:
            raise ArticleKB_ProcessSoupError(url = self.url, search_term= search_term)

        tarticle = []
        for row in table:
            # print("❤️")

            cells = row.find(class_="slds-form-element__label")

            if list(cells.strings):
                content = row.find(class_="slds-form-element__control")
                tarticle.append((list(cells.strings)[0], content))

        kb_soup = dict(tarticle)
        self.kb_soup = kb_soup

        self.title = self.article.md_soup(kb_soup.get("Title"))

        self.md_str = self.article.md_soup(kb_soup.get("Article Body"))
        self.article_id = self.article.md_soup(kb_soup.get("Article Number"))
        self.views = self.article.md_soup(kb_soup.get("Article Total View Count"))
        self.created = parser.parse(
            self.article.md_soup(kb_soup.get("Article Created Date"))
        )

        self.last_updated = parser.parse(
            self.article.md_soup(kb_soup.get("First Published Date"))
        )

        self.get_images(
            test_base_url='https://domo-support.domo.com//servlet/rtaImage')

        return self.kb_soup

test_broken_url = 'https://domo-support.domo.com/s/knowledge-base/'
test_cls = Article_KB(url=URL, base_url=BASE_URL, driver=driver)

💤 loading https://domo-support.domo.com/s/article/360047400753?language=en_US 💤
Page https://domo-support.domo.com/s/article/360047400753?language=en_US is loaded within 10 seconds.


In [9]:
# test_cls.kb_soup


In [10]:
import mdutils
import os
from mdutils.mdutils import MdUtils
import requests

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()

def add_frontmatter( data, file_path: str ):
    with open(file_path, 'r+', encoding='utf-8') as md_file:
        file_data = md_file.read()  # Save all the file's content
        md_file.seek(0, 0)  # Place file pointer at the beginning
        md_file.write(data)  # Write data
        md_file.write('\n' + file_data)

def download_img(image_url, image_path):
    img_data = requests.get(image_url).content
    with open(image_path, 'wb') as handler:
        handler.write(img_data)

def output_md(
    article: Article_KB, output_folder: str = "../kb_md", debug_prn: bool = False, is_create_subfolder: bool = True
):

    output_path = os.path.join(output_folder, convert_to_snake(article.title))

    if debug_prn:
        print(f"outputting {article.title} to {output_path}")

    output_index = output_path

    if is_create_subfolder:
        is_path_exist = os.path.exists(output_path)

        if not is_path_exist:
            os.makedirs(output_path)
        
        for img in article.image_ls:
            download_img(image_url=img.get('url'),
                         image_path=f"{output_path}\{img.get('name')}")
            
            print(img.get('relative_url'))
            article.md_str = article.md_str.replace(img.get('relative_url'), img.get('name'))
            

        output_index = f"{output_path}\index"

    md_file = MdUtils(file_name=output_index)
    
    md_file.write(article.md_str)

    md_file.create_md_file()

    frontmatter_obj = f"""---
    title: {article.title}
    url: {article.url}
    linked_kbs:  { [ md_file.new_inline_link(link) for link in article.kb_url_ls]}
    article_id: {article.article_id}
    views: {article.views}
    created_date: {str(article.created)}
    last updated: {str(article.last_updated)}
    ---"""

    add_frontmatter(data=frontmatter_obj, file_path=f"{output_path}.md")

output_md(article = test_cls, debug_prn= True)

outputting Jupyter Workspaces to ../kb_md\jupyter_workspaces
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkK
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkP
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkL
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkS
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkS
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkS
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkS
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkR
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkX
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkI
/servlet/rtaImage?eid=ka05w00000125tb&feoid=00N5w00000Ri7BU&refid=0EM5w000005vXkV
/servlet/rtaImage?eid=ka05w00000125tb

FileNotFoundError: [Errno 2] No such file or directory: '../kb_md\\jupyter_workspaces.md'

In [ ]:
import logging

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)


class Crawler:
    base_url: str
    output_folder: str
    urls_visited_ls: list[str]
    urls_to_vist_ls: list[str]

    driver: selenium.webdriver

    def __init__(self,
                 urls_to_visit_ls: list[str] = None,
                 base_url=None,
                 output_folder='kb_md',
                 ):

        self.base_url = base_url
        self.output_folder = output_folder
        self.urls_visited_ls = []
        self.urls_to_visit_ls = urls_to_visit_ls
        self.article_ls = []
        self.driver = dcc.driversetup(is_headless=False)

    def add_url_to_visit(self, url, debug_prn: bool = False):
        if url not in self.urls_visited_ls and url not in self.urls_to_visit_ls:
            if debug_prn:
                print(f'adding url to list - {url}')

            self.urls_to_visit_ls.append(url)

    def crawl(self, url, debug_prn: bool = False):
        if debug_prn:
            print(f"starting crawl - {url}")

        article = Article_KB(url=url, base_url=self.base_url, driver=driver)

        for url in article.kb_url_ls:
            self.add_url_to_visit(url=url, debug_prn=debug_prn)
        
        if article.is_success:
            output_md(article=article, output_folder=self.output_folder,
                    debug_prn=debug_prn)


    def run(self, debug_prn: bool = False):
        while self.urls_to_visit_ls:
            url = self.urls_to_visit_ls.pop(0)

            logging.info(f'Crawling: {url}')

            try:
                self.crawl(url, debug_prn)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.urls_visited_ls.append(url)

        print('done')
        return self


In [ ]:
crawler = Crawler(urls_to_visit_ls=[URL], base_url=BASE_URL, output_folder="kb_md")

crawler.run(debug_prn=True)

2023-02-23 23:10:48,049 INFO:Crawling: https://domo-support.domo.com/s/article/360047400753?language=en_US
2023-02-23 23:10:48,053 ERROR:Failed to crawl: https://domo-support.domo.com/s/article/360047400753?language=en_US
Traceback (most recent call last):
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_36792\2042628069.py", line 57, in run
    self.crawl(url, debug_prn)
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_36792\2042628069.py", line 40, in crawl
    article = Article_KB(url=url, base_url=self.base_url, driver=driver)
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_36792\1665088112.py", line 44, in __init__
    soup = dcc.pagesource(driver=self.driver, url=self.url,
  File "c:\Users\jwilson1\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\crawler.py", line 94, in pagesource
    driver.get(url)
  File "c:\Users\jwilson1\Miniconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 449, in get
    self.execute(Command.GET, {"url": ur

starting crawl - https://domo-support.domo.com/s/article/360047400753?language=en_US
💤 loading https://domo-support.domo.com/s/article/360047400753?language=en_US 💤
done
